In [223]:
import cv2
import numpy as np
from time import sleep

In [224]:
width_min=80 #length of the rectangle
height_min=80 #height of the rectangle

In [225]:
offset=10 #Error allowed between pixel

pos_line=675 #Position of line at the bottom

delay= 60 #FPS of vídeo

detec = []
carros= 0

In [226]:
def center(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy

In [227]:
cap = cv2.VideoCapture('ch01_20181127091944.mp4')
sub = cv2.bgsegm.createBackgroundSubtractorMOG()

#cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
#cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

#frame_width = int(cap.get(3))
#frame_height = int(cap.get(4))
 
#Define the codec and create VideoWriter object.The output is stored in 'output.avi' file.
#out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
#firstFrame = None

In [228]:
while True:
    ret , frame = cap.read()
    tempo = float(1/delay)
    sleep(tempo) 
    grey = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey,(3,3),5)
    img_sub = sub.apply(blur)
    dilat = cv2.dilate(img_sub,np.ones((5,5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilatada = cv2.morphologyEx (dilat, cv2. MORPH_CLOSE , kernel)
    dilatada = cv2.morphologyEx (dilatada, cv2. MORPH_CLOSE , kernel)
    
    img,contours,h = cv2.findContours(dilatada,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cv2.line(frame, (25, pos_line), (1500, pos_line), (255,127,0), 3) 
    for(i,c) in enumerate(contours):
        (x,y,w,h) = cv2.boundingRect(c)
        validate_contours = (w >= width_min) and (h >= height_min)
        if not validate_contours:
            continue

        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)        
        centroid = center(x, y, w, h)
        detec.append(centroid)
        cv2.circle(frame, centroid, 4, (0, 0,255), -1)

        for (x,y) in detec:
            if y<(pos_line+offset) and y>(pos_line-offset):
                carros+=1
                cv2.line(frame, (15, pos_line), (1200, pos_line), (0,127,255), 3)  
                detec.remove((x,y))
                print("Cars detected so far: "+str(carros))        
       
    cv2.putText(frame, "Vehicle Count: "+str(carros), (450, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255),5)
    cv2.imshow("Video Original" , frame)
    cv2.imshow("Detector",dilatada)
    
    if cv2.waitKey(27)  & 0xFF == ord('q'):
        break
    
   # if ret == True:
    #    print('writing')
    #out.write(frame)
    
    
cv2.destroyAllWindows()
#out.release()
cap.release()

Cars detected so far: 1
Cars detected so far: 2
Cars detected so far: 3
